# Binary Text Classification of sentence Question or Not Question

In [59]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import warnings  
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(".../input/binary_dataset.csv")

In [3]:
df.drop("Unnamed: 2", axis=1, inplace=True)
df.head()

,Text,Class
0,how to register to vote,Question
1,how to tie a tie,Question
2,can you run it,Question
3,what song is this,Question
4,how to lose weight,Question


In [4]:
df = df[df['Text'].notna()]
df = df[df['Class'].notna()]

In [5]:
df.shape

(3979, 2)

In [7]:
X = df.Text
Y = df.Class
le = LabelEncoder()
Y = le.fit_transform(Y)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

## 1: Not Question, 2: Question

In [17]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [24]:
model = XGBClassifier()
model.fit(X_train, Y_train)

[11:24:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [26]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.23%


In [57]:
s = ["Do you want to play a game"]
test = vectorizer.transform(s)
test_pred = model.predict(test)

In [58]:
test_pred


array([2])

In [40]:
model.save_model('question_notQuestion.json')

In [43]:
import pickle

pickle.dump(vectorizer, open("question_vectors.pickle", "wb"))
pickle.dump(model, open("question_notQuestion_model.pickle", "wb"))


In [52]:
s = ["Hey Bro How are you?"]
infile = open("question_vectors.pickle",'rb')
vec = pickle.load(infile)
test = vec.transform(s)
infile1 = open("question_notQuestion_model.pickle",'rb')
mod = pickle.load(infile1)
test_pred = mod.predict(test)

In [54]:
test_pred[0]

2

In [ ]:
def load_models(vectorizer_path, model_path):
    vec_file = open(vectorizer_path,'rb')
    vectorizer = pickle.load(vec_file)

    model_file = open(model_path,'rb')
    model = pickle.load(model_file)
    
    return vectorizer, model


def isQuestion(text, vactorizer, model):
    text = [text]
    text = vactorizer.transform(text)
    pred = model.predict(text)
    if pred[0] == 2:
        return True
    else:
        return False